In [1]:
using CSV, Tables
using JuMP
using Gurobi
using JSON3
using JSON
using HTTP

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Set parameter LicenseID to value 2595637
Academic license - for non-commercial use only - expires 2025-12-05


#### Load in Data

In [3]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [4]:
random_class = all_classes[rand(1:length(all_classes))]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating             => 4.9
  :has_final          => true
  :description        => "Subset of elementary discrete mathematics for science…
  :offered_fall       => false
  :offered_spring     => true
  :instructors        => ["P. Jaillet"]
  :schedule           => "Lecture,32-144/TR/0/1-2.30;Recitation,13-3101/WF/0/1,…
  :out_of_class_hours => 8.5
  :url                => "http://student.mit.edu/catalog/m6a.html#6.120A"
  :total_units        => 6
  :related_subjects   => ["6.1200", "18.062", "6.5060", "6.7480", "6.5340", "6.…
  :pdf_option         => false
  :in_class_hours     => 4.6
  :is_half_class      => false
  :level              => "U"
  :prerequisites      => "GIR:CAL1"
  :subject_id         => "6.120A"
  :title              => "Discrete Mathematics and Proof for Computer Science"
  :lab_units          => 0
  ⋮                   => ⋮

In [5]:
subjectid_to_index = Dict(class[:subject_id] => ind for (ind, class) in enumerate(all_classes))
index_to_subjectid = Dict(ind => class[:subject_id] for (ind, class) in enumerate(all_classes))

Dict{Int64, String} with 4704 entries:
  4700 => "WGS.301"
  4576 => "STS.022"
  2288 => "21G.905"
  1703 => "18.C20"
  1956 => "20.101"
  2350 => "21H.226"
  3406 => "6.2060"
  2841 => "24.234"
  2876 => "24.906"
  687  => "12.751"
  185  => "1.THG"
  1090 => "15.518"
  2015 => "20.560"
  3293 => "5.00"
  1704 => "18.S096"
  3220 => "4.608"
  422  => "11.353"
  1266 => "16.004"
  183  => "1.S992"
  ⋮    => ⋮

#### Defining the Basic Optimization Problem

In [6]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [7]:
@variable(model, X[1:n, 1:S], Bin)

4704×8 Matrix{VariableRef}:
 X[1,1]     X[1,2]     X[1,3]     …  X[1,6]     X[1,7]     X[1,8]
 X[2,1]     X[2,2]     X[2,3]        X[2,6]     X[2,7]     X[2,8]
 X[3,1]     X[3,2]     X[3,3]        X[3,6]     X[3,7]     X[3,8]
 X[4,1]     X[4,2]     X[4,3]        X[4,6]     X[4,7]     X[4,8]
 X[5,1]     X[5,2]     X[5,3]        X[5,6]     X[5,7]     X[5,8]
 X[6,1]     X[6,2]     X[6,3]     …  X[6,6]     X[6,7]     X[6,8]
 X[7,1]     X[7,2]     X[7,3]        X[7,6]     X[7,7]     X[7,8]
 X[8,1]     X[8,2]     X[8,3]        X[8,6]     X[8,7]     X[8,8]
 X[9,1]     X[9,2]     X[9,3]        X[9,6]     X[9,7]     X[9,8]
 X[10,1]    X[10,2]    X[10,3]       X[10,6]    X[10,7]    X[10,8]
 ⋮                                ⋱  ⋮                     
 X[4696,1]  X[4696,2]  X[4696,3]  …  X[4696,6]  X[4696,7]  X[4696,8]
 X[4697,1]  X[4697,2]  X[4697,3]     X[4697,6]  X[4697,7]  X[4697,8]
 X[4698,1]  X[4698,2]  X[4698,3]     X[4698,6]  X[4698,7]  X[4698,8]
 X[4699,1]  X[4699,2]  X[4699,3]     X[4699,

In [8]:
α = 1 # alpha between [0-1], 1 being fully optimizing on minimizing hours 
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

In [9]:
# Full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# Wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# Not repeatable: cannot take a class again
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1);\

# Taken correct semester
@constraint(model, [s in 1:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_fall]);
@constraint(model, [s in 2:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_spring]);

# 2 CI-H or HW
@constraint(model, sum(
    (get(all_classes[i], :communication_requirement, "") in ["CI-H", "CI-HW"]) * X[i, s] for i=1:n, s=1:S) >= 2);

# 8 HASS
@constraint(model, sum(
    (get(all_classes[i], :hass_attribute, "") in ["HASS-H", "HASS-A", "HASS-S"]) * X[i, s] for i=1:n, s=1:S) >= 8);

# 1 of each HASS-H, A, S
@constraint(model, [hass_type in ["HASS-H", "HASS-A", "HASS-S"]], sum(
    (get(all_classes[i], :hass_attribute, "") in [hass_type]) * X[i, s] for i=1:n, s=1:S) >= 1);


#### Adding Institute Reqs ####

In [10]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_inds_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2963, 3304, 3305]
 [3857, 3858, 3859, 3860]
 [3861, 3862, 3863]
 [1537, 1538]
 [1539, 1541, 1540]
 [3753, 3754, 3755, 3756, 3757]

In [11]:
# GIR constraints
@constraint(model, [gir_type in gir_inds_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1);

#### Adding 6-4 Constraints ####

In [12]:
# Store the URL
url = "https://fireroad.mit.edu/requirements/get_json/major6-4/"

# Send the request and get the response
reqs = HTTP.get(url)

# Parse the JSON response
datafile = JSON.parse(String(reqs.body))

Dict{String, Any} with 7 entries:
  "short-title"     => "6-4"
  "medium-title"    => "6-4 Major"
  "title-no-degree" => "Artificial Intelligence and Decision Making"
  "reqs"            => Any[Dict{String, Any}("connection-type"=>"all", "thresho…
  "list-id"         => "major6-4.reql"
  "title"           => "Bachelor of Science in Artificial Intelligence and Deci…
  "desc"            => "The Bachelor of Science in Artificial Intelligence and …

In [13]:
# Create a mapping for comparison types
type_mapping = Dict("LT" => "lt", "GT" => "gt", "GTE" => "geq", "LTE" => "leq")

# Function to recursively go through requirements and create vector of major requirements
function create_requirements(data, connector=false, cutoff=1, ineq="geq")
    mandatory = []
    all_reqs = []
    together = false
    comb_req = []
    
    for section in data["reqs"]
        ineq = "geq"
        cutoff = 1
        
        # Base case: if there's a "req" key in the section
        if haskey(section, "req")
            if connector
                push!(mandatory, section["req"])
                continue
            else
                push!(all_reqs, section["req"])
                continue
            end
        end

        # Recursive case: checking for special thresholds
        if haskey(section, "distinct-threshold")
            cutoff = section["distinct-threshold"]["cutoff"]
            ineq = type_mapping[section["distinct-threshold"]["type"]]
            if haskey(section, "threshold") && section["threshold"]["cutoff"] != cutoff
                together = true
                total_cutoff = section["threshold"]["cutoff"]
                total_ineq = type_mapping[section["threshold"]["type"]]
            end
        elseif haskey(section, "threshold")
            cutoff = section["threshold"]["cutoff"]
            ineq = type_mapping[section["threshold"]["type"]]
        end
        
        # Building based on connection types
        if section["connection-type"] == "all"
            next = create_requirements(section, true)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                append!(all_reqs, next)
            end
        elseif section["connection-type"] == "any"
            next = create_requirements(section)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                one_req = (next, ineq, cutoff)
                push!(all_reqs, one_req)
            end
        else
            return create_requirements(section)
        end
        
        #for thresholds with distinct-thresholds
        if together
            push!(all_reqs, (comb_req, total_ineq, total_cutoff))
        end
    end

    if !isempty(mandatory)
        push!(all_reqs, (mandatory, "eq", length(mandatory)))
    end
    
    return all_reqs
end

# Call the function
requirements = create_requirements(datafile)

10-element Vector{Any}:
 (Any["6.100A", "6.100L"], "geq", 1)
 (Any["6.S084", "18.C06", "18.06"], "geq", 1)
 (Any["6.3700", "6.3800", "18.05"], "geq", 1)
 (Any["6.1200", "6.1010", "6.1210"], "eq", 3)
 (Any[(Any["6.3720", "6.3900", (Any["6.C01", "6.S052"], "eq", 2), (Any["6.C51", "6.S952"], "eq", 2), "6.S059"], "geq", 1), (Any["6.3000", "6.4110", "6.4400"], "geq", 1), (Any["6.3100", "6.4110", "6.7201"], "geq", 1), (Any["6.1220", "6.4400", "6.7201"], "geq", 1), (Any["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "6.S041", "9.660"], "geq", 1)], "geq", 5)
 (Any["6.1800", "6.2040", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"], "geq", 2)
 (Any["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"], "geq", 1)
 (Any["6.4200", "6.4210", "6.8301", "6.8611"], "geq", 1)
 (Any["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.7411", "6.7930", "6.8300", "6.8301", "6.8371", "6.8611", "6.870

In [74]:
for req in requirements
    courses, ineq, cutoff = req
    if any(x -> isa(x, Tuple), courses)
        println(courses)
    end
end

Any[(Any["6.3720", "6.3900", (Any["6.C01", "6.S052"], "eq", 2), (Any["6.C51", "6.S952"], "eq", 2), "6.S059"], "geq", 1), (Any["6.3000", "6.4110", "6.4400"], "geq", 1), (Any["6.3100", "6.4110", "6.7201"], "geq", 1), (Any["6.1220", "6.4400", "6.7201"], "geq", 1), (Any["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "6.S041", "9.660"], "geq", 1)]
Any[(Any["18.600", "2.007", (Any["6.100A", "6.100B"], "eq", 2), "6.1010", "6.1020", "6.1040", "6.1060", "6.1100", "6.1120", "6.1200", "6.1210", "6.1220", "6.1400", "6.1600", "6.1800", "6.1820", "6.1850", "6.1910", "6.1920", "6.2000", "6.2040", "6.2050", "6.2060", "6.2061", "6.2090", "6.2200", "6.2210", "6.2220", "6.2221", "6.2300", "6.2370", "6.2400", "6.2410", "6.2500", "6.2530", "6.2600", "6.2690", "6.3000", "6.3010", "6.3100", "6.3260", "6.3400", "6.3700", "6.3702", "6.3730", "6.3800", "6.3900", "6.4100", "6.4120", "6.4130", "6.4140", "6.4200", "6.4210", "6.4400", "6.4420", "6.4510", "6.4530", "6.4550", "6.4570", "6.4590", "6.4710", "6.4810",

In [ ]:
requirements[5]

10-element Vector{Any}:
 (Any["6.100A", "6.100L"], "geq", 1)
 (Any["6.S084", "18.C06", "18.06"], "geq", 1)
 (Any["6.3700", "6.3800", "18.05"], "geq", 1)
 (Any["6.1200", "6.1010", "6.1210"], "eq", 3)
 (Any[(Any["6.3720", "6.3900", (Any["6.C01", "6.S052"], "eq", 2), (Any["6.C51", "6.S952"], "eq", 2), "6.S059"], "geq", 1), (Any["6.3000", "6.4110", "6.4400"], "geq", 1), (Any["6.3100", "6.4110", "6.7201"], "geq", 1), (Any["6.1220", "6.4400", "6.7201"], "geq", 1), (Any["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "6.S041", "9.660"], "geq", 1)], "geq", 5)
 (Any["6.1800", "6.2040", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"], "geq", 2)
 (Any["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"], "geq", 1)
 (Any["6.4200", "6.4210", "6.8301", "6.8611"], "geq", 1)
 (Any["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.7411", "6.7930", "6.8300", "6.8301", "6.8371", "6.8611", "6.870

In [53]:
# Helper function to map the inequality string to a JuMP constraint
function create_inequality_constraint(model, expr, ineq::String, value)
    if ineq == "geq"
        @constraint(model, expr >= value)
    elseif ineq == "eq"
        @constraint(model, expr == value)
    elseif ineq == "leq"
        @constraint(model, expr <= value)
    else
        error("Unknown inequality type: $ineq")
    end
end

# Main function to create JuMP constraints based on the requirements vector
function add_requirements_constraints(model::Model, requirements::Vector, bin_var::Bool=false, group::Bool=false)
    outer_group = Vector{Any}()
    for req in requirements
        courses, ineq, cutoff = req

        # If the course requirement is a nested structure (i.e., a vector of other requirements)
        if any(x -> isa(x, Tuple), courses)
            # Recursively handle the nested requirements
            solos = Vector{Any}()
            for nested_req in courses
                if isa(nested_req, Tuple)
                    temps = add_requirements_constraints(model, [nested_req], true)
                    append!(outer_group, temps)
                else
                    push!(solos, nested_req)
                end
            end
            if !isempty(solos)
                temps = add_requirements_constraints(model, [(solos, "geq", 1)], true)
                append!(outer_group, temps)
            end
            final_tup = (outer_group, ineq, cutoff)
            add_requirements_constraints(model, [final_tup], false, true)
            outer_group = [@variable(model, binary=true)]
        else
            # Retrieve the variables for each course in the list
            if group
                course_inds = courses
                course_expr = sum(course_inds)
            else
                course_inds = [subjectid_to_index[course] for course in courses if haskey(subjectid_to_index, course)]
                if isempty(course_inds)
                    continue
                end
                course_expr = sum(X[course_ind, s] for course_ind in course_inds, s in 1:S)
            end

            if bin_var
                inner_group = @variable(model, binary=true)
                push!(outer_group, inner_group)
                # Create the appropriate inequality constraint
                new_cutoff = cutoff * inner_group
                create_inequality_constraint(model, course_expr, ineq, new_cutoff)
            else
                # Create the appropriate inequality constraint
                create_inequality_constraint(model, course_expr, ineq, cutoff)
            end
        end
    end
    return outer_group
end

real_requirements = [(requirements, "eq", length(requirements))]
add_requirements_constraints(model, real_requirements)

1-element Vector{VariableRef}:
 _[37999]

#### Adding Prereq constraints

In [147]:
raw_prerequisites = map(course -> get(course, "prerequisites", nothing), all_classes)

4703-element Vector{Union{Nothing, String}}:
 "GIR:CAL1"
 nothing
 "GIR:CAL1"
 nothing
 nothing
 nothing
 nothing
 "GIR:CAL2"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [148]:
abstract type ExprNode end

struct CourseNode <: ExprNode
    course::String
end

struct ANDNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct ORNode <: ExprNode
    left::ExprNode
    right::ExprNode
end

struct PreReq
    root::CourseNode
    prereqs::Union{ExprNode, Nothing}
end 

In [149]:
function parse_prereq_string(input)
    if input === nothing
        return
    end
    input = replace(input, r"\s+" => "")
    tokens = collect(input)
    function parse(tokens::Vector{Char})::ExprNode
        stack = ExprNode[]
        buffer = IOBuffer()
        while !isempty(tokens)
            token = popfirst!(tokens)
            if token == '('
                node = parse(tokens)
                push!(stack, node)
            elseif token == ')'
                break
            elseif token == ','
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ANDNode(left, right)
            elseif token == '/'
                left = isempty(stack) ? CourseNode(String(take!(buffer))) : pop!(stack)
                right = parse(tokens)
                return ORNode(left, right)
            else
                write(buffer, token)
            end
        end
        return isempty(stack) ? CourseNode(String(take!(buffer))) : stack[1]
    end
    return parse(tokens)
end

parse_prereq_string (generic function with 1 method)

In [150]:
function replace_gir_terms(input_str)
    if input_str === nothing
        return 
    end
    gir_map = Dict(
        "GIR:CHEM" => ["3.091", "5.111", "5.112"],
        "GIR:PHY1" => ["8.01", "8.011", "8.012", "8.01L"],
        "GIR:PHY2" => ["8.02", "8.021", "8.022"],
        "GIR:CAL1" => ["18.01", "18.01A"],
        "GIR:CAL2" => ["18.02", "18.02A", "18.022"],
        "GIR:BIOL" => ["7.012", "7.013", "7.014", "7.015", "7.016"]
    )
    pattern = r"GIR:\w+"
    replaced_str = replace(input_str, pattern => x -> begin
        term = match(pattern, x).match
        if haskey(gir_map, term)
            "(" * join(gir_map[term], "/") * ")"
        else
            term
        end
    end)
    return replaced_str
end

replace_gir_terms (generic function with 1 method)

In [151]:
prerequisites = map(prereq_str -> replace_gir_terms(prereq_str), raw_prerequisites)

4703-element Vector{Union{Nothing, String}}:
 "(18.01/18.01A)"
 nothing
 "(18.01/18.01A)"
 nothing
 nothing
 nothing
 nothing
 "(18.02/18.02A/18.022)"
 nothing
 "''Permission of instructor''"
 "2.003, 2.016, 2.678"
 nothing
 nothing
 ⋮
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 nothing
 "''One intermediate subject in French''/''permission of instructor''"
 nothing

In [152]:
prerequisite_expressions = []
for index in 1:length(prerequisites)
    course_id = CourseNode(index_to_subjectid[index])
    prereq_tree = parse_prereq_string(prerequisites[index])
    push!(prerequisite_expressions, PreReq(course_id, prereq_tree))
end
prerequisite_expressions

4703-element Vector{Any}:
 PreReq(CourseNode("1.00"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.000"), nothing)
 PreReq(CourseNode("1.001"), ORNode(CourseNode("18.01"), CourseNode("18.01A")))
 PreReq(CourseNode("1.005"), nothing)
 PreReq(CourseNode("1.007"), nothing)
 PreReq(CourseNode("1.008"), nothing)
 PreReq(CourseNode("1.009"), nothing)
 PreReq(CourseNode("1.010"), ORNode(CourseNode("18.02"), ORNode(CourseNode("18.02A"), CourseNode("18.022"))))
 PreReq(CourseNode("1.011"), nothing)
 PreReq(CourseNode("1.013"), CourseNode("''Permissionofinstructor''"))
 PreReq(CourseNode("1.015"), ANDNode(CourseNode("2.003"), ANDNode(CourseNode("2.016"), CourseNode("2.678"))))
 PreReq(CourseNode("1.016"), nothing)
 PreReq(CourseNode("1.018"), nothing)
 ⋮
 PreReq(CourseNode("WGS.250"), nothing)
 PreReq(CourseNode("WGS.270"), nothing)
 PreReq(CourseNode("WGS.271"), nothing)
 PreReq(CourseNode("WGS.274"), nothing)
 PreReq(CourseNode("WGS.275"), nothing)
 PreReq(CourseNod

In [153]:
function add_course_constraint(course::CourseNode, model::Model, sem::Int)
    index = get(subjectid_to_index, course.course, nothing)
    if index == nothing
        return 0  # Ignore unknown courses
    end
    if sem == 1
        return 0  # Cannot satisfy prerequisites in the first semester
    else
        return handle_prerequisite_sum(index, model, sem)
    end
end

function handle_prerequisite_sum(index::Int, model::Model, sem::Int)
    prereq_sum = sum(X[index, s] for s in 1:sem-1)
    binary_prereq_var = @variable(model, binary=true)
    @constraint(model, binary_prereq_var <= prereq_sum)
    @constraint(model, binary_prereq_var >=  prereq_sum / (sem-1))  # Ensure binary behavior
    return binary_prereq_var
end

function add_and_constraint(left_var, right_var, model::Model)
    and_var = @variable(model, binary=true)
    @constraint(model, and_var <= left_var)
    @constraint(model, and_var <= right_var)
    @constraint(model, and_var >= left_var + right_var - 1)
    return and_var
end

function add_or_constraint(left_var, right_var, model::Model)
    or_var = @variable(model, binary=true)
    @constraint(model, or_var >= left_var)
    @constraint(model, or_var >= right_var)
    @constraint(model, or_var <= left_var + right_var)
    return or_var
end

function add_and_or_constraints(expr::ExprNode, model::Model, sem::Int)
    if isa(expr, CourseNode)
        return add_course_constraint(expr, model, sem)
    elseif isa(expr, ANDNode)
        left_var = add_and_or_constraints(expr.left, model, sem)
        right_var = add_and_or_constraints(expr.right, model, sem)
        return add_and_constraint(left_var, right_var, model)
    elseif isa(expr, ORNode)
        left_var = add_and_or_constraints(expr.left, model, sem)
        right_var = add_and_or_constraints(expr.right, model, sem)
        return add_or_constraint(left_var, right_var, model)
    else
        error("Unknown expression type")
    end
end

function add_prereq_constraint(prereq_obj::PreReq, model::Model, sem::Int)
    index = get(subjectid_to_index, prereq_obj.root.course, nothing)
    if index == nothing
        return 1  # Ignore unknown courses
    end
    if prereq_obj.prereqs === nothing
        return X[index, sem]
    else
        prereq_var = add_and_or_constraints(prereq_obj.prereqs, model, sem)
        @constraint(model, X[index, sem] <= prereq_var)
        return X[index, sem]
    end
end


add_prereq_constraint (generic function with 1 method)

In [154]:
### Adding Course prereq constraints
for s in 1:S
    for prereq_expr in prerequisite_expressions
        add_prereq_constraint(prereq_expr, model, s)
    end
end

#### Optimizing the Model and Interpretation

In [155]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 21.5.0 21F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 422334 rows, 155174 columns and 1359208 nonzeros
Model fingerprint: 0x50040df8
Variable types: 0 continuous, 155174 integer (155174 binary)
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e+00, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve produced solution with objective 217.68 (0.18s)
Loaded MIP start from previous solve with objective 217.68

Presolve removed 377967 rows and 129679 columns
Presolve time: 2.47s
Presolved: 44367 rows, 25495 columns, 153568 nonzeros
Variable types: 0 continuous, 25495 integer (25495 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing primal log only...

Concurrent spin time: 0.03s

Solved with dual simplex

Use crossover to convert LP s

In [162]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end



function interpret_course_attr(classes_id_per_sem, attr)
    courses_attr = [[get(all_classes[class_ind], Symbol(attr), "") for class_ind in sem] for sem in classes_id_per_sem]
    return courses_attr
end

function interpret_course_hours(classes_id_per_sem)
    course_hours = [[H[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_hours
end

function interpret_course_units(classes_id_per_sem)
    course_units = [[U[class_ind] for class_ind in sem] for sem in classes_id_per_sem]
    return course_units
end

interpret_course_units (generic function with 1 method)

In [161]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["21M.622", "21M.747", "8.01L"]
 ["18.01", "21M.460", "3.091", "6.100A"]
 ["18.022", "6.1010", "6.1200"]
 ["21M.623", "6.1210", "ES.113"]
 ["21M.451", "21W.032", "6.3700", "8.02"]
 ["18.06", "6.1400", "6.3260"]
 ["6.3900", "6.3950", "7.012"]
 ["6.3100", "6.8301", "6.UAT"]

In [166]:
function interpret_course_attr(classes_id_per_sem, "title")

LoadError: UndefVarError: `course_ids` not defined

In [159]:
prerequisite_expressions[subjectid_to_index["16.84"]]

PreReq(CourseNode("16.84"), ORNode(CourseNode("6.4200"), CourseNode("''permissionofinstructor''")))

In [160]:
sems_hours = interpret_course_hours(taken_classes_sem)
[sum(sem_hours) for sem_hours in sems_hours]

8-element Vector{Float64}:
 20.32
 31.049999999999997
 31.43
 20.43
 31.09
 28.15
 25.86
 29.349999999999998